In [1]:
import random
import pandas as pd
from datetime import datetime

# Temporary workflow
# (1) Download csv with `heroku run make exportdb && make importdb`
# (2) save links to ~/Downloads/links.txt in onetab format
# (3) run this script (or otherwise modify the CSV as desired)
# (4) check date cell
# (5) Upload back up with `make csv_to_s3 && heroku run make importdb`
links = pd.read_csv('data/export.csv')
links

/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87838,149331,https://clarifyingconsequences.substack.com/p/...,A nuclear war forecast is not a coin flip,"When projecting risk over a time period\, ecau...",clarifyingconsequences.substack.com,2022-05-14 00:56:57.839619+00,2022-05-14 00:56:57.839613+00,1.0,xrisk,Custom,1.0,0.0,0
87839,149332,https://marginalrevolution.com/marginalrevolut...,Against credentialism,"Sure credentials are helpful\, but not perfect...",marginalrevolution.com,2022-05-14 00:58:47.702101+00,2022-05-14 00:58:47.702095+00,1.0,management,Custom,1.0,0.0,0
87840,149333,https://www.philvenables.com/post/human-error,Human error,"""In every incident or close-call [...] where t...",www.philvenables.com,2022-05-14 00:59:13.88144+00,2022-05-14 00:59:13.881433+00,1.0,technology,Custom,1.0,0.0,0
87841,149334,https://dynomight.net/teaching/,Teaching is a slow process of becoming everyth...,Four stories from teaching about how terrible-...,dynomight.net,2022-05-14 00:59:45.874112+00,2022-05-14 00:59:45.874103+00,1.0,policy,Custom,1.0,0.0,0


In [2]:
links['added'].max()

'2022-05-14 01:04:32.452885+00'

In [3]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]['id']

/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)


1299       1306
1300       1307
1301       1308
1302       1309
1303       1310
          ...  
87838    149331
87839    149332
87840    149333
87841    149334
87842    149335
Name: id, Length: 7785, dtype: int64

In [4]:
links[links['url'].duplicated()]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
87828,149320,https://www.centreforeffectivealtruism.org/blo...,CEA’s 2017 review and 2018 plans,\N,\N,2022-05-13 15:18:26.452107+00,2022-05-13 15:18:26.452107+00,NaN,\N,CEA,40.0,0.0,\N
87838,149331,https://clarifyingconsequences.substack.com/p/...,A nuclear war forecast is not a coin flip,"When projecting risk over a time period\, ecau...",clarifyingconsequences.substack.com,2022-05-14 00:56:57.839619+00,2022-05-14 00:56:57.839613+00,1.0,xrisk,Custom,1.0,0.0,0


In [5]:
links[links['url'].duplicated()]['id']

87828    149320
87838    149331
Name: id, dtype: int64

In [6]:
link_file = open('/Users/peterhurford/Downloads/links.txt', 'r')
new_links = link_file.readlines()

In [7]:
def new_entry(links, url, title):
    return {'id': links['id'].max() + 1,
            'url': str(url).strip().replace(',', ''),
            'title': str(title).strip().replace(',', ''),
            'summary': '\\N',
            'domain': '\\N',
            'added': str(datetime.now()),
            'modified': str(datetime.now()),
            'liked': '\\N',
            'category': '\\N',
            'aggregator': 'Custom',
            'seed': random.randint(1, 100),
            'tweet': 0}

for i, new_link in enumerate(new_links):
    if i % 10 == 0 or i == len(new_links) - 1:
        print('...{}/{}'.format(i + 1, len(new_links)))
    if ' | ' in new_link:
        new_link = new_link.replace('    - ', '').replace('\n', '')
        entry = new_entry(links, new_link.split(' | ')[0], new_link.split(' | ')[1])
        links = links.append(entry, ignore_index=True)

links

...1/261
...11/261
...21/261
...31/261
...41/261
...51/261
...61/261
...71/261
...81/261
...91/261
...101/261
...111/261
...121/261
...131/261
...141/261
...151/261
...161/261
...171/261
...181/261
...191/261
...201/261
...211/261
...221/261
...231/261
...241/261
...251/261
...261/261


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88097,149590,https://atis.substack.com/p/the-case-against-p...,The Case Against Public Intellectuals - by Sam...,\N,\N,2022-05-13 20:32:16.768072,2022-05-13 20:32:16.768084,\N,\N,Custom,70.0,0.0,NaN
88098,149591,https://atis.substack.com/p/what-happens-when-...,What happens when we pay MPs more? - by Sam Atis,\N,\N,2022-05-13 20:32:16.806048,2022-05-13 20:32:16.806060,\N,\N,Custom,55.0,0.0,NaN
88099,149592,https://plato.stanford.edu/entries/computation...,The Computational Theory of Mind (Stanford Enc...,\N,\N,2022-05-13 20:32:16.843996,2022-05-13 20:32:16.844008,\N,\N,Custom,54.0,0.0,NaN
88100,149593,https://arxiv.org/abs/2202.07206,[2202.07206] Impact of Pretraining Term Freque...,\N,\N,2022-05-13 20:32:16.883040,2022-05-13 20:32:16.883052,\N,\N,Custom,17.0,0.0,NaN


In [8]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
1299,1306,http://blog.beeminder.com/trust/,Trusting Your Divided Self | Beeminder Blog,\N,blog.beeminder.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,productivity,Custom,10.0,0.0,0
1300,1307,http://powlyglot.com/fi3m-review-part-1-focus/,"""What I Learned from Fluent in 3 Months\, Part...",\N,powlyglot.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,productivity,Custom,9.0,0.0,0
1301,1308,http://effective-altruism.com/ea/u8/ea_intervi...,"""EA Interview Series\, February 2016: John Sal...",\N,effective-altruism.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,effectivealtruism,Custom,85.0,0.0,0
1302,1309,https://fivethirtyeight.com/features/most-2020...,Most 2020 Candidates Have Something In Common:...,\N,fivethirtyeight.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,1,politicalscience,Custom,82.0,0.0,0
1303,1310,http://www.smartpassiveincome.com/backlinking-...,THE Backlinking Strategy That Works – 2014 and...,\N,www.smartpassiveincome.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,entrepreneurship,Custom,59.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88097,149590,https://atis.substack.com/p/the-case-against-p...,The Case Against Public Intellectuals - by Sam...,\N,\N,2022-05-13 20:32:16.768072,2022-05-13 20:32:16.768084,\N,\N,Custom,70.0,0.0,NaN
88098,149591,https://atis.substack.com/p/what-happens-when-...,What happens when we pay MPs more? - by Sam Atis,\N,\N,2022-05-13 20:32:16.806048,2022-05-13 20:32:16.806060,\N,\N,Custom,55.0,0.0,NaN
88099,149592,https://plato.stanford.edu/entries/computation...,The Computational Theory of Mind (Stanford Enc...,\N,\N,2022-05-13 20:32:16.843996,2022-05-13 20:32:16.844008,\N,\N,Custom,54.0,0.0,NaN
88100,149593,https://arxiv.org/abs/2202.07206,[2202.07206] Impact of Pretraining Term Freque...,\N,\N,2022-05-13 20:32:16.883040,2022-05-13 20:32:16.883052,\N,\N,Custom,17.0,0.0,NaN


In [9]:
def clean(txt):
    if not isinstance(txt, str):
        return txt
    txt = txt.replace('&nbsp;', '')
    txt = txt.replace('&ldquo;', '"')
    txt = txt.replace('&rdquo;', '"')
    txt = txt.replace('&lsquo;', '\'')
    txt = txt.replace('&rsquo;', '\'')
    txt = txt.replace('&mdash;', '-')
    txt = txt.replace('&ndash;', '-')
    for i in range(10):
        txt = txt.replace('"""', '"')
        txt = txt.replace('\'\'\'', '\'')
    txt = txt.replace('n"t', 'n\'t')
    txt = txt.replace('n""t', 'n\'t')
    return txt

links = links[links['id'].notnull()]   # Drop empty column
links['id'] = links['id'].astype(int)  # Fix float ID issue
links['seed'] = links['seed'].astype(int)
links['tweet'] = links['tweet'].apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int)
links['summary'] = links['summary'].apply(clean)
links['title'] = links['title'].apply(clean)
links = links.sort_values('id')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1,ethics,custom,17,0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1,giving,custom,59,0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1,productivity,custom,56,0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1,politicalscience,custom,29,0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1,technology,custom,92,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88097,149590,https://atis.substack.com/p/the-case-against-p...,The Case Against Public Intellectuals - by Sam...,\N,\N,2022-05-13 20:32:16.768072,2022-05-13 20:32:16.768084,\N,\N,Custom,70,0,NaN
88098,149591,https://atis.substack.com/p/what-happens-when-...,What happens when we pay MPs more? - by Sam Atis,\N,\N,2022-05-13 20:32:16.806048,2022-05-13 20:32:16.806060,\N,\N,Custom,55,0,NaN
88099,149592,https://plato.stanford.edu/entries/computation...,The Computational Theory of Mind (Stanford Enc...,\N,\N,2022-05-13 20:32:16.843996,2022-05-13 20:32:16.844008,\N,\N,Custom,54,0,NaN
88100,149593,https://arxiv.org/abs/2202.07206,[2202.07206] Impact of Pretraining Term Freque...,\N,\N,2022-05-13 20:32:16.883040,2022-05-13 20:32:16.883052,\N,\N,Custom,17,0,NaN


In [10]:
links.to_csv('data/export.csv', index=False)